In [189]:
import os
from pathlib import Path
from typing import Any
from pydantic import Field, BaseModel
import httpx

from grasp_agents import (
    BaseTool,
    LLMAgent,
    RunContext,
    ImageData,
    Printer,
    print_event_stream,
    ParallelProcessor,
)
from grasp_agents.runner import Runner
from grasp_agents.typing.events import ProcPacketOutEvent
from grasp_agents.openai import OpenAILLM, OpenAILLMSettings
from grasp_agents.litellm import LiteLLM, LiteLLMSettings
from grasp_agents.workflow.sequential_workflow import SequentialWorkflow
from grasp_agents.cloud_llm import APIProvider
from grasp_agents.rate_limiting import RateLimiter

from grasp_agents.telemetry.traceloop import init_traceloop

# Optional: enable LLM observability with Phoenix
from grasp_agents.telemetry.phoenix import init_phoenix

In [190]:
PACKAGE_DIR = Path.cwd()

Paths to images used in the demo

In [191]:
IMG_1_URL = "https://www.simplilearn.com/ice9/free_resources_article_thumb/Expressions_In_C_2.PNG"
IMG_2_PATH = PACKAGE_DIR / "src/grasp_agents/examples/data/expr.jpeg"

Utils

In [192]:
def print_single_output(out: Any) -> None:
    print(f"\n<final answer>\n{out.payloads[0]}\n</final answer>")

**[Optional] Observability with Arize Phoenix (deployed locally via Docker)**

Start Phoenix locally:
```bash
cd ./phoenix
docker compose up -d  
docker compose logs -f phoenix
export PHOENIX_COLLECTOR_HTTP_ENDPOINT=http://localhost:6006/v1/traces # from docker-compose.yml
export TELEMETRY_PROJECT_NAME_KEY="openinference.project.name" # required by Phoenix
```

Open `http://localhost:6006/` (or the port you set in `docker-compose.yml`) in your browser to access the Phoenix UI.

Initialize telemetry

In [193]:
# Use Traceloop to produce spans (without sending them anywhere yet)
init_traceloop(project_name="agents-demo")

# Use Phoenix as the backend and UI for telemetry
# Alternatively, any OpenTelemetry-compatible backend can be used instead
init_phoenix(batch=False, use_litellm_instr=True, use_llm_provider_instr=False)

Attempting to instrument while already instrumented


## Simple generation with validated outputs

Output type validation

In [194]:
# list[int] is the output type used to validate the output
chatbot = LLMAgent[None, list[int], None](
    name="chatbot",
    llm=LiteLLM(
        model_name="gpt-4o-mini",
        llm_settings=LiteLLMSettings(logprobs=True),
        max_response_retries=2,
    ),
)

# This initialises printer and usage tracker
ctx = RunContext[None](printer=Printer())

10:45:32 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
10:45:32 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
10:45:32 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
10:45:32 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30


In [195]:
# Code block delimiters are stripped from the output
out = await chatbot.run(
    "Output a list of 3 integers from 0 to 10 as a python array, no talking",
    ctx=ctx,
)
print_single_output(out)

<chatbot> [f8fbcc_chatbot]
<input>
Output a list of 3 integers from 0 to 10 as a python array, no talking
</input>

<chatbot> [f8fbcc_chatbot]
<response>
```python
[3, 7, 1]
```
</response>

------------------------------------
I/O/R/C tokens: 30/13/-/-


<final answer>
[3, 7, 1]
</final answer>


In [196]:
ctx.usage_tracker.usages

{'chatbot': Usage(input_tokens=30, output_tokens=13, reasoning_tokens=0, cached_tokens=0, cost=1.23e-05)}

Completion data (e.g. log probs) per agent can be accessed via RunContext:

In [ ]:
# ctx.completions

Streaming

In [197]:
chatbot = LLMAgent[None, list[int], None](
    name="chatbot",
    llm=LiteLLM(
        model_name="claude-sonnet-4-20250514",
        llm_settings=LiteLLMSettings(reasoning_effort=None),
    ),
    stream_llm_responses=True,
)
ctx = RunContext[None](printer=None)

10:45:38 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
10:45:38 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
10:45:38 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
10:45:38 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30


In [198]:
async for event in print_event_stream(
    chatbot.run_stream(
        "Output a list of 30 integers from 0 to 10 as a python array. "
        "No code or talking.",
        ctx=ctx,
    )
):
    if isinstance(event, ProcPacketOutEvent):
        out = event.data


<chatbot> [bf2da7_chatbot]
<input>
Output a list of 30 integers from 0 to 10 as a python array. No code or talking.
</input>

<chatbot> [bf2da7_chatbot]
<response>
[7, 2, 9, 1, 5, 8, 3, 6, 0, 4, 9, 2, 7, 5, 1, 8, 6, 3, 10, 4, 0, 9, 7, 2, 5, 8, 1, 6, 4, 3]
</response>

<chatbot> [bf2da7_chatbot]
<processor output>
[
  7,
  2,
  9,
  1,
  5,
  8,
  3,
  6,
  0,
  4,
  9,
  2,
  7,
  5,
  1,
  8,
  6,
  3,
  10,
  4,
  0,
  9,
  7,
  2,
  5,
  8,
  1,
  6,
  4,
  3
]
</processor output>


In [199]:
ctx.usage_tracker.usages

{'chatbot': Usage(input_tokens=33, output_tokens=93, reasoning_tokens=0, cached_tokens=0, cost=0.0014939999999999999)}

Output type validation with structured outputs

In [200]:
# Some providers (e.g. `openai` and `gemini`) support structured outputs.
# With the OpenAI API, this will require a Pydantic model to validate the output.

from enum import StrEnum


class Selector(StrEnum):
    A = "A"
    B = "B"


class Response(BaseModel):
    result: list[int] = Field(..., description="3 random integers")
    value: Selector = Field(..., description="Choose a value randomly")


chatbot = LLMAgent[None, Response, None](
    name="chatbot",
    response_schema=Response,
    llm=LiteLLM(
        model_name="gemini/gemini-2.5-flash",
        llm_settings=LiteLLMSettings(),
        apply_response_schema_via_provider=True,
    ),
)

# By default, response_schema is set to the output type of the agent (Response)
# In some cases, you may want to set it to a different type, e.g. when using
# custom output parsing.

ctx = RunContext[None](printer=Printer())

10:45:47 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
10:45:47 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
10:45:47 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
10:45:47 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30


In [201]:
out = await chatbot.run("start", ctx=ctx)
print_single_output(out)

<chatbot> [3c51f1_chatbot]
<input>
start
</input>

<chatbot> [3c51f1_chatbot]
<response>
{
  "result": [
    14,
    76,
    3
  ],
  "value": "A"
}
</response>

------------------------------------
I/O/R/C tokens: 2/27/69/-


<final answer>
result=[14, 76, 3] value=<Selector.A: 'A'>
</final answer>


# Chat with images

In [202]:
chatbot = LLMAgent[None, str, None](
    name="chatbot",
    llm=LiteLLM(
        model_name="gemini/gemini-2.5-flash",
        llm_settings=LiteLLMSettings(reasoning_effort="disable"),
    ),
    sys_prompt="You are a helpful assistant",
)
ctx = RunContext[None](printer=Printer())

10:45:53 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
10:45:53 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
10:45:53 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
10:45:53 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30


In [203]:
out = await chatbot.run("Where are you headed, stranger?", ctx=ctx)
print_single_output(out)

<chatbot> [6d337b_chatbot]
<system>
You are a helpful assistant
</system>

<chatbot> [6d337b_chatbot]
<input>
Where are you headed, stranger?
</input>

<chatbot> [6d337b_chatbot]
<response>
I am a large language model, trained by Google.
</response>

------------------------------------
I/O/R/C tokens: 13/11/-/-


<final answer>
I am a large language model, trained by Google.
</final answer>


In [204]:
out = await chatbot.run("What did you just say, exactly?", ctx=ctx)
print_single_output(out)

<chatbot> [174b16_chatbot]
<input>
What did you just say, exactly?
</input>

<chatbot> [174b16_chatbot]
<response>
I said, "I am a large language model, trained by Google."
</response>

------------------------------------
I/O/R/C tokens: 34/15/-/-


<final answer>
I said, "I am a large language model, trained by Google."
</final answer>


In [205]:
out = await chatbot.run(
    ["What's in this image?", ImageData.from_path(IMG_2_PATH)], ctx=ctx
)
print_single_output(out)

<chatbot> [15292b_chatbot]
<input>
What's in this image?
<ENCODED_IMAGE>
</input>

<chatbot> [15292b_chatbot]
<response>
The image displays a mathematical expression: **7 * (5 + 15) / (2 * 5) - 3**.
</response>

------------------------------------
I/O/R/C tokens: 316/28/-/-


<final answer>
The image displays a mathematical expression: **7 * (5 + 15) / (2 * 5) - 3**.
</final answer>


In [206]:
out = await chatbot.run("Do the calculation", ctx=ctx)
print_single_output(out)

<chatbot> [8b5a4c_chatbot]
<input>
Do the calculation
</input>

<chatbot> [8b5a4c_chatbot]
<response>
Let's break down the calculation step by step, following the order of operations (PEMDAS/BODMAS):

1.  **Parentheses/Brackets:**
    *   (5 + 15) = 20
    *   (2 * 5) = 10

2.  **Substitute these back into the expression:**
    7 * 20 / 10 - 3

3.  **Multiplication and Division (from left to right):**
    *   7 * 20 = 140
    *   140 / 10 = 14

4.  **Substitute this back into the expression:**
    14 - 3

5.  **Subtraction:**
    *   14 - 3 = 11

So, the result of the calculation is **11**.
</response>

------------------------------------
I/O/R/C tokens: 349/187/-/-


<final answer>
Let's break down the calculation step by step, following the order of operations (PEMDAS/BODMAS):

1.  **Parentheses/Brackets:**
    *   (5 + 15) = 20
    *   (2 * 5) = 10

2.  **Substitute these back into the expression:**
    7 * 20 / 10 - 3

3.  **Multiplication and Division (from left to right):**
    

In [207]:
out = await chatbot.run(["Try another one", ImageData.from_url(IMG_1_URL)], ctx=ctx)
print_single_output(out)

<chatbot> [faa8ff_chatbot]
<input>
Try another one
https://www.simplilearn.com/ice9/free_resources_article_thumb/Expressions_In_C_2.PNG
</input>

<chatbot> [faa8ff_chatbot]
<response>
This image presents an arithmetic expression with given variable values.

**Arithmetic Expression:**
Z = a + b - (a * c)

**Given values:**
a = 2
b = 3
c = 4

Let's calculate the value of Z:

1.  **Substitute the values into the expression:**
    Z = 2 + 3 - (2 * 4)

2.  **Perform the operation inside the parentheses first:**
    (2 * 4) = 8

3.  **Substitute the result back into the expression:**
    Z = 2 + 3 - 8

4.  **Perform addition and subtraction from left to right:**
    *   2 + 3 = 5
    *   5 - 8 = -3

Therefore, Z = **-3**.
</response>

------------------------------------
I/O/R/C tokens: 799/180/-/-


<final answer>
This image presents an arithmetic expression with given variable values.

**Arithmetic Expression:**
Z = a + b - (a * c)

**Given values:**
a = 2
b = 3
c = 4

Let's calculate the 

In [208]:
out = await chatbot.run("What was my first question, exactly?", ctx=ctx)
print_single_output(out)

<chatbot> [f8a07b_chatbot]
<input>
What was my first question, exactly?
</input>

<chatbot> [f8a07b_chatbot]
<response>
Your first question was: "Where are you headed, stranger?"
</response>

------------------------------------
I/O/R/C tokens: 224/13/-/765


<final answer>
Your first question was: "Where are you headed, stranger?"
</final answer>


In [209]:
ctx.usage_tracker.total_usage

Usage(input_tokens=1735, output_tokens=434, reasoning_tokens=None, cached_tokens=765, cost=0.0011640500000000002)

# Parallel runs with retries and rate limiting

In [210]:
# Make the LLM generate text instead of integers occasionally
# to emphasise the need for retries

sys_prompt = """
You are a bad math student who always adds number {added_num} to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
"""

in_prompt = "What is the square of {num}?"


class RunArgs(BaseModel):
    added_num: int


class InputArgs(BaseModel):
    num: int


# Specifying int as the output type means that the agent will
# validate the output against this type.

student = LLMAgent[InputArgs, int, RunArgs](
    name="student",
    llm=LiteLLM(
        model_name="gpt-4.1",
        llm_settings=LiteLLMSettings(temperature=1.8),
        # This rate limit will be applied to parallel runs of the agent
        rate_limiter=RateLimiter(rpm=100),
    ),
    sys_prompt=sys_prompt,
    in_prompt=in_prompt,
    reset_memory_on_run=True,
    max_retries=2,
)


@student.add_system_prompt_builder
def system_prompt_builder(ctx: RunContext[RunArgs], **kwargs: Any) -> str:
    return student.sys_prompt.format(added_num=ctx.state.added_num)

10:46:12 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
10:46:12 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
10:46:12 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
10:46:12 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30


In [211]:
in_args = [InputArgs(num=i) for i in range(10)]

In [212]:
ctx = RunContext[RunArgs](state=RunArgs(added_num=5), printer=Printer())

# Wrap the agent in a ParallelProcessor to run it in parallel
out = await ParallelProcessor(student).run(in_args=in_args, ctx=ctx)

print()
print(*[p for p in out.payloads], sep="\n")

<student> [60e2df_student_par/0]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
</system>

<student> [60e2df_student_par/0]
<input>
What is the square of 0?
</input>

<student> [60e2df_student_par/1]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
</system>

<student> [60e2df_student_par/1]
<input>
What is the square of 1?
</input>

<student> [60e2df_student_par/2]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
</system>

<student> [60e2df_student_par/2]
<input>
What is the square of 2?
</input>

<student> [60e2df_student_par/3]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer o

In [213]:
ctx = RunContext[RunArgs](state=RunArgs(added_num=5))

# Do not stream granular LLM events here
stream = ParallelProcessor(student).run_stream(in_args=in_args, ctx=ctx)
async for event in print_event_stream(stream):
    pass


<student> [04456e_student_par/0]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
</system>

<student> [04456e_student_par/0]
<input>
What is the square of 0?
</input>

<student> [04456e_student_par/1]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
</system>

<student> [04456e_student_par/1]
<input>
What is the square of 1?
</input>

<student> [04456e_student_par/2]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
</system>

<student> [04456e_student_par/2]
<input>
What is the square of 2?
</input>

<student> [04456e_student_par/3]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer 

Both ast.literal_eval and json.loads failed to parse the following JSON/Python string:
Six
Processor run failed [proc_name=student; call_id=04456e_student_par/1] -> retrying (attempt 1):
Failed to validate LLM response:
Six
Expected type: <class 'int'>



<student> [04456e_student_par/1]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
</system>

<student> [04456e_student_par/1]
<input>
What is the square of 1?
</input>


Both ast.literal_eval and json.loads failed to parse the following JSON/Python string:
Nine
Processor run failed [proc_name=student; call_id=04456e_student_par/2] -> retrying (attempt 1):
Failed to validate LLM response:
Nine
Expected type: <class 'int'>



<student> [04456e_student_par/2]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
</system>

<student> [04456e_student_par/2]
<input>
What is the square of 2?
</input>


Both ast.literal_eval and json.loads failed to parse the following JSON/Python string:
Thirty	
Processor run failed [proc_name=student; call_id=04456e_student_par/5] -> retrying (attempt 1):
Failed to validate LLM response:
Thirty	
Expected type: <class 'int'>



<student> [04456e_student_par/5]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
</system>

<student> [04456e_student_par/5]
<input>
What is the square of 5?
</input>

<student_par> [04456e_student_par]
<processor output>
5
6
9
14
21
30
41
54
69
86
</processor output>


# Reasoning agent loop with streaming

In [214]:
sys_prompt_react = """
Your task is to suggest an exciting stats problem to the student. 
You should first ask the student about their education, interests, and preferences, then suggest a problem tailored specifically to them. 

# Instructions
* Use the provided tool to ask questions.
* Ask questions one by one.
* The problem must have all the necessary data.
* Use the final answer tool to provide the problem.
"""

In [215]:
# Tool input must be a Pydantic model to infer the JSON schema used by the LLM APIs
class TeacherQuestion(BaseModel):
    question: str


StudentReply = str


ask_student_tool_description = """
"Ask the student a question and get their reply."

Args:
    question: str
        The question to ask the student.
Returns:
    reply: str
        The student's reply to the question.
"""


class AskStudentTool(BaseTool[TeacherQuestion, StudentReply, Any]):
    name: str = "ask_student"
    description: str = ask_student_tool_description

    async def run(self, inp: TeacherQuestion, **kwargs: Any) -> StudentReply:
        return input(inp.question)

In [216]:
class Problem(BaseModel):
    problem: str


teacher = LLMAgent[None, Problem, None](
    name="teacher",
    llm=LiteLLM(
        model_name="claude-sonnet-4-5",
        llm_settings=LiteLLMSettings(reasoning_effort="low"),
    ),
    tools=[AskStudentTool()],
    # react_mode=True, # use with non-reasoning models to enforce preamble/tool/response structure
    final_answer_as_tool_call=True,
    sys_prompt=sys_prompt_react,
    stream_llm_responses=True,
    tracing_exclude_input_fields={"ctx"},
)

10:47:03 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
10:47:03 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
10:47:03 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
10:47:03 - LiteLLM:WARNING: logging_callback_manager.py:138 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30


TypeError: LLMAgent.__init__() got an unexpected keyword argument 'tracing_exclude_input_fields'

In [ ]:
ctx = RunContext[None]()

events = []
problem: Problem
async for event in print_event_stream(teacher.run_stream("start", ctx=ctx)):
    if isinstance(event, ProcPacketOutEvent):
        problem = event.data.payloads[0]
    events.append(event)

In [ ]:
problem

# Sequential workflow 

In [ ]:
# Input arguments are passed to the agent dynamically (e.g. by other agents)
from grasp_agents.typing.content import Content


# Global state is used to store data that is shared between runs of the agent.
class State(BaseModel):
    b: int
    c: int


class AddInputArgs(BaseModel):
    a: int = Field(..., description="First number to add.")


class AddResponse(BaseModel):
    a_plus_b: int


add_in_prompt = "Add {a} and {b}. Your only output is the resulting number."


add_agent = LLMAgent[AddInputArgs, AddResponse, State](
    name="add_agent",
    llm=LiteLLM(model_name="gpt-4.1"),
    in_prompt=add_in_prompt,
    # Reset message history to system prompt (if provided) before each run
    reset_memory_on_run=True,
    stream_llm_responses=True,
)


@add_agent.add_input_content_builder
def build_input_content_impl(
    in_args: AddInputArgs, ctx: RunContext[State], call_id: str
) -> Content:
    return Content.from_formatted_prompt(
        add_agent.in_prompt, a=in_args.a, b=ctx.state.b
    )


@add_agent.add_output_parser
def parse_output_impl(
    final_answer: str,
    *,
    in_args: AddInputArgs | None = None,
    ctx: RunContext[State],
    call_id: str,
) -> AddResponse:
    return AddResponse(a_plus_b=int(final_answer.strip()))

In [ ]:
class MultiplyResponse(BaseModel):
    c_a_plus_b: int


multiply_in_prompt = (
    "Multiply {a_plus_b} by {c}. Your only output is the resulting number."
)

multiply_agent = LLMAgent[AddResponse, MultiplyResponse, State](
    name="multiply_agent",
    llm=LiteLLM(model_name="gpt-4.1"),
    in_prompt=multiply_in_prompt,
    reset_memory_on_run=True,
    stream_llm_responses=True,
)


# Need a custom input content maker to use the global state
@multiply_agent.add_input_content_builder
def build_input_content_impl(
    in_args: AddResponse, ctx: RunContext[State], call_id: str
) -> Content:
    return Content.from_formatted_prompt(
        multiply_agent.in_prompt, a_plus_b=in_args.a_plus_b, c=ctx.state.c
    )


@multiply_agent.add_output_parser
def parse_output_impl(
    final_answer: str,
    *,
    in_args: AddResponse | None = None,
    ctx: RunContext[State],
    call_id: str,
) -> MultiplyResponse:
    return MultiplyResponse(c_a_plus_b=int(final_answer.strip()))

In [ ]:
seq_agent = SequentialWorkflow[AddInputArgs, MultiplyResponse, State](
    name="seq_agent", subprocs=[add_agent, multiply_agent]
)

In [ ]:
state = State(b=3, c=6)
ctx = RunContext[State](state=state, printer=Printer())

In [ ]:
out = await seq_agent.run(in_args=AddInputArgs(a=2), ctx=ctx)

# out = await ParallelProcessor(seq_agent).run(
#     in_args=[AddInputArgs(a=2), AddInputArgs(a=3)], ctx=ctx
# )

# Agents as tools

When agents are used as tools, their `in_args` become the tool inputs.

This is how one can implement a manager + helpers architecture.

In [ ]:
seq_tool = seq_agent.as_tool(
    tool_name="seq_agent_tool",
    tool_description=(
        "A sequential agent that adds 3 to a given integer, "
        "then multiplies the result by 5."
    ),
)

The JSON schema of `in_args` is preserved:

In [ ]:
seq_tool.in_type.model_json_schema()

In [ ]:
await seq_tool(a=15, ctx=ctx)

Stream from the tool

In [ ]:
state = State(b=3, c=6)
ctx = RunContext[State](state=state)

seq_agent = SequentialWorkflow[AddInputArgs, MultiplyResponse, State](
    name="seq_agent", subprocs=[add_agent, multiply_agent]
)
seq_tool = seq_agent.as_tool(
    tool_name="seq_agent_tool",
    tool_description=(
        "A sequential agent that adds 3 to a given integer, "
        "then multiplies the result by 5."
    ),
)
stream = seq_tool.run_stream(AddInputArgs(a=15), ctx=ctx)

async for event in print_event_stream(stream):
    pass

# Teacher / students

A more advanced example of multi-agent debate, where agents communicate using the actor model.

Communication schemas

In [ ]:
from collections.abc import Sequence
from typing import Literal


TeacherRecipient = Literal["*END*", "teacher", "student1", "student2"]


# Teacher can choose which students to send the message to
class TeacherExplanation(BaseModel):
    explanation: str
    selected_recipients: Sequence[TeacherRecipient] = Field(
        default_factory=list[TeacherRecipient],
        description="Recipients selected by the teacher.",
    )


# Students can only ask questions to the teacher
class StudentQuestion(BaseModel):
    question: str = Field(
        ...,
        description="The question to ask the teacher.",
    )

#### Teacher

In [ ]:
teacher_sys_prompt = """
You are a teacher explaining quantum gravity to a 2-year old baby (named 'student1') and a 30-year old graphic designer (named student2). 
Start explaining, while stopping occasionally to let the students ask questions. 
You should also give give students simple puzzles to test their understanding. 
Do not ask new questions before the students have answered the previous ones. 
To indicate to whom you are addressing your message, you must specify the recipients as a list of selected student names. 
When students have no more questions, finish the conversation with a SINGLE message with a SINGLE recipient called *END*. 
Do not produce multiple "thanks" or "goodbye" messages, just a single one.
"""

teacher = LLMAgent[StudentQuestion, TeacherExplanation, None](
    name="teacher",
    llm=LiteLLM(model_name="gpt-4o", apply_response_schema_via_provider=True),
    sys_prompt=teacher_sys_prompt,
    # need to specify allowed recipients to choose from
    recipients=["*END*", "student1", "student2"],
)


@teacher.add_recipient_selector
def select_recipients_impl(
    output: TeacherExplanation, **kwargs: Any
) -> Sequence[TeacherRecipient] | None:
    return output.selected_recipients

#### Students

In [ ]:
student_sys_prompts = [
    """
You are a 4-year old child trying to make sense of physics. 
Your name is <student1>.
Talk to the teacher to understand the topic.
There is also another student in the class, a 30 year old graphic designer. 
You talk to the teacher only.
""",
    """
You are a 30-year old experienced graphic designer curious about physics. 
Your name is <student2>.
Ask questions to the teacher until you understand the topic. 
Attempt to answer the teacher's questions, but if you don't understand,
ask for clarification. 
There is also another student in the class, a 4-year old child.
You talk to the teacher only.
""",
]


def make_student_agent(name: str, sys_prompt: str):
    student = LLMAgent[TeacherExplanation, StudentQuestion, None](
        name=name,
        llm=LiteLLM(model_name="gpt-4o", apply_response_schema_via_provider=True),
        sys_prompt=sys_prompt,
        recipients=["teacher"],
    )

    @student.add_output_parser
    def parse_output_impl(final_answer: str, **kwargs: Any) -> StudentQuestion:
        return StudentQuestion(question=f"<{name}>: " + str(final_answer))

    return student


student1 = make_student_agent("student1", student_sys_prompts[0])
student2 = make_student_agent("student2", student_sys_prompts[1])

In [ ]:
ctx = RunContext[None](printer=Printer(color_by="agent"))
runner = Runner(entry_proc=teacher, procs=[teacher, student1, student2], ctx=ctx)
final_result = await runner.run("start")

Streaming

In [ ]:
# runner = Runner(
#     entry_proc=teacher, procs=[teacher, student1, student2], ctx=RunContext[None]()
# )
# events = []
# async for event in print_event_stream(
#     runner.run_stream(chat_inputs="start"), color_by="agent"
# ):
#     events.append(event)

# Custom API providers and HTTP clients

In [ ]:
custom_provider = APIProvider(
    name="openrouter",
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
)

http_client = httpx.AsyncClient(
    timeout=httpx.Timeout(10),
    limits=httpx.Limits(max_connections=10),
)

chatbot = LLMAgent[None, list[int], None](
    name="chatbot",
    llm=OpenAILLM(
        model_name="deepseek/deepseek-r1-0528",
        api_provider=custom_provider,
        http_client=http_client,
    ),
)


ctx = RunContext[None](printer=Printer())
out = await chatbot.run(
    "Output a list of 3 integers from 0 to 10 as a python array, no talking",
    ctx=ctx,
)
print_single_output(out)